## Theano Introduction
0. Linear Regression
1. Simple Model
    * Dense layer implementation
    * Test using some data set
    * Dropout layer
2. Convolution
    * Convolution 2D layer
    * Test using keras 2d data set

In [117]:
from keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [82]:
#So theano is doing symbolic computation
#for my dense layer I want to define my input array 
#(lets say this is a 64 neuron dense layer)
import theano
from theano import shared
import theano.tensor as T
import theano.tensor.nnet as N
import math
import numpy as np
from theano import config

theano.config.exception_verbosity='high'

#taken from fast.ai lesson6 notebook
def init_wgts(rows, cols): 
    scale = math.sqrt(2/rows)
    return shared(np.random.normal(scale=scale, size=(rows, cols)).astype(np.float32),name='lw')
def init_bias(rows): 
    return shared(np.zeros(rows, dtype=np.float32), name='lb')

In [104]:
inp = T.vector('inp') # input array of weights (batch size 1)
weights = T.matrix('wei') # array of weights per neuron 
biases = T.vector('bia') # array of biases per neuron
tar = T.scalar('tar') #target 
lr = T.scalar('lr') # learning rate
gradient_weights = T.matrix('gw')
gradient_biases = T.vector('bw')

#shared variables
layer_weights= init_wgts(len(x_train[0]),64)
layer_biases = init_bias(64)

#def step(x, weights,biases):
#    return N.relu(x.dot(weights) + biases)
def step(x, weights, biases):
    return N.relu(x.dot(layer_weights) + layer_biases)

#for each neuron we want to do the activation(input * weights + bias)
#so i think that the issue lies in my use of the scan function.  does not seem as if I'm doing what I think I am
#is calculating a dense layer a single matrix multiplication?
#results, updates = theano.scan(step, sequences=inp, 
#                              non_sequences=[layer_weights,layer_biases])
results = step(inp,layer_weights,layer_biases)

summed = T.sum(results)
error = T.sqrt(T.sqr(summed-tar))

#How to calculate gradient?
gy = T.grad(error, [layer_weights, layer_biases])

In [144]:
theano.printing.debugprint(gycalc)
max(y_train)

GpuGer{inplace=True} [id A] ''   15
 |GpuAlloc<None>{memset_0=True} [id B] ''   6
 | |GpuArrayConstant{0.0} [id C]
 | |Shape_i{0} [id D] ''   1
 | | |inp [id E]
 | |Shape_i{1} [id F] ''   3
 |   |lw [id G]
 |TensorConstant{1.0} [id H]
 |GpuFromHost<None> [id I] ''   0
 | |inp [id E]
 |GpuElemwise{Composite{(i0 + ((i1 * i2 * sgn(i3)) / i4))}}[(0, 3)]<gpuarray> [id J] ''   14
   |GpuElemwise{Composite{((i0 * i1) / i2)}}[]<gpuarray> [id K] ''   13
   | |GpuArrayConstant{[ 0.5]} [id L]
   | |GpuElemwise{Composite{((i0 * i1) - i2)}}[(0, 1)]<gpuarray> [id M] ''   11
   | | |GpuArrayConstant{[ 0.5]} [id L]
   | | |InplaceGpuDimShuffle{x} [id N] ''   10
   | | | |GpuCAReduceCuda{add} [id O] ''   9
   | | |   |GpuElemwise{Composite{(i0 + Abs(i0))}}[]<gpuarray> [id P] ''   8
   | | |     |GpuGemv{inplace=False} [id Q] ''   5
   | | |       |lb [id R]
   | | |       |TensorConstant{1.0} [id H]
   | | |       |InplaceGpuDimShuffle{1,0} [id S] ''   2
   | | |       | |lw [id G]
   | | |       |GpuF

50.0

In [119]:
gycalc = theano.function([inp,tar],gy,allow_input_downcast=True)
step_cal = theano.function([inp], results, allow_input_downcast=True)

In [120]:
#why does gradient return zeros?
gycalc(x_train[10],y_train[10])

[gpuarray.array([[  -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,   -4.7978549 ,   -4.7978549 ,   -4.7978549 ,
           -4.7978549 ,  

In [124]:
from collections import *
#update function?
#dimensionality different for what i want vs what jeremy did
#what i had before was probably right (tuple form)
def upd_dict(weights, biases, grads, lr):
    return [(weights , weights + -1 * lr * grads[0]),
                        (biases, biases + -1 * lr * grads[1])]
upd = upd_dict(layer_weights, layer_biases, gy, lr)


def upd_dict_test(weights, biases, grads, lr):
    return weights + -1 * lr * grads[0]

test_upd = upd_dict_test(layer_weights, layer_biases, gy, lr)

In [125]:
upd_f = theano.function([inp,tar,lr], test_upd, allow_input_downcast=True)

In [126]:
upd_f(x_train[10], y_train[10], .01)

array([[ 0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855,  0.04797855,
         0.04797855,  0.04797855,  0.04797855,  0.04797855],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.

In [127]:
#goal try to write a function for this update
#so my problem was not using shared variables for state
#T.grad also needs to use shared variables
#fn = theano.function([inp,weights,biases,lr], error, updates=upd)
fn = theano.function([inp,tar,lr], error, updates=upd,allow_input_downcast=True)
pred = theano.function([inp], summed, allow_input_downcast=True)

In [140]:
for x,y in zip(x_train,y_train):
    fn(x,y,.01)


In [129]:
#theano.printing.debugprint(fn)
#for all x_test, y_test
# get error and then we can take the rms of it
s = 0.0
for x,y in zip(x_test,y_test):
    s = s + math.sqrt((y-pred(x))**2)
print(s)



2354.0


In [132]:
pred([1,1,1,1,1,1,1,1,1,1,1,1,1])
#theano.printing.debugprint(pred)

array(0.0, dtype=float32)

In [139]:
print(layer_weights.get_value())

[[  5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03   5.94465015e-03   5.94465015e-03   5.94465015e-03
    5.94465015e-03  

In [133]:
theano.printing.debugprint(pred)

HostFromGpu(gpuarray) [id A] ''   6
 |GpuElemwise{Mul}[(0, 1)]<gpuarray> [id B] ''   5
   |GpuArrayConstant{0.5} [id C]
   |GpuCAReduceCuda{add} [id D] ''   4
     |GpuElemwise{Composite{(i0 + Abs(i0))}}[(0, 0)]<gpuarray> [id E] ''   3
       |GpuGemv{inplace=False} [id F] ''   2
         |lb [id G]
         |TensorConstant{1.0} [id H]
         |InplaceGpuDimShuffle{1,0} [id I] ''   1
         | |lw [id J]
         |GpuFromHost<None> [id K] ''   0
         | |inp [id L]
         |TensorConstant{1.0} [id H]
